In [ ]:
# from pybiomart import Dataset as PyBiomartDataset

# dataset = PyBiomartDataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')
# genes_proteins = dataset.query(attributes=['ensembl_gene_id', 'ensembl_peptide_id',
#                                             'gene_biotype', 'transcript_is_canonical', 'peptide'], filters={})
# # genes_proteins = genes_proteins[genes_proteins['Ensembl Canonical'] == 1.0]
# genes_proteins.to_csv('gene2protein.csv')

In [5]:
import pandas as pd

In [6]:
genes_proteins = pd.read_csv('gene2protein.csv', index_col=0)
genes_proteins = genes_proteins[genes_proteins['Ensembl Canonical'] == 1.0]

In [23]:
genes_proteins = genes_proteins.set_index('Gene stable ID')

In [24]:
genes_proteins

,Peptide,Protein stable ID,Gene type,Ensembl Canonical
Gene stable ID,,,,
ENSG00000001036,MRPQELPRLAFPLLLLLLLLLPPPPCPAHSATRFDPTWESLDARQL...,ENSP00000002165,protein_coding,1.0
ENSG00000001626,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,ENSP00000003084,protein_coding,1.0
ENSG00000003137,MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...,ENSP00000001146,protein_coding,1.0
ENSG00000004799,MKAARFVLRSAGSLNGAGLVPREVEHFSRYSPSPLSMKQLLDFGSE...,ENSP00000005178,protein_coding,1.0
ENSG00000004059,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ENSP00000000233,protein_coding,1.0
...,...,...,...,...
ENSG00000286080,Sequence unavailable,NaN,lncRNA,1.0
ENSG00000290192,Sequence unavailable,NaN,lncRNA,1.0
ENSG00000291307,MEPKARCSDSQRGGPCTNHEPPALEEGGRRHRRARAGSSSGALGLQ...,ENSP00000516626,protein_coding,1.0


In [31]:
from os.path import join


datapath = "/home/ec2-user/cytoself-data"

df = pd.read_csv(join(datapath, "labels.csv"), index_col=0)
# df.to_csv(join(datapath, "labels.csv"))

In [28]:
# df['Peptide'] = df['ensg'].map(genes_proteins['Peptide'])
# df['Protein stable ID'] = df['ensg'].map(genes_proteins['Protein stable ID'])
# df.to_csv(join(datapath, "labels.csv"))

In [37]:
prots = df[['ensg', 'Peptide']].drop_duplicates().set_index('ensg')
prots.to_csv(join(datapath, "sequences.csv"))

In [45]:
prots['Peptide'].str.find('*')

ensg
ENSG00000075624     375
ENSG00000100591     338
ENSG00000156735     457
ENSG00000229524    1132
ENSG00000105401     378
                   ... 
ENSG00000174749     132
ENSG00000138439     589
ENSG00000116001     386
ENSG00000108588     483
ENSG00000125912     563
Name: Peptide, Length: 1311, dtype: int64

In [47]:
prots['Peptide'].iloc[0]

'MDDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVGDEAQSKRGILTLKYPIEHGIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETFNTPAMYVAIQAVLSLYASGRTTGIVMDSGDGVTHTVPIYEGYALPHAILRLDLAGRDLTDYLMKILTERGYSFTTTAEREIVRDIKEKLCYVALDFEQEMATAASSSSLEKSYELPDGQVITIGNERFRCPEALFQPSFLGMESCGIHETTFNSIMKCDVDIRKDLYANTVLSGGTTMYPGIADRMQKEITALAPSTMKIKIIAPPERKYSVWIGGSILASLSTFQQMWISKQEYDESGPSIVHRKCF*'

## Load embeddings

In [49]:
GENE2PROTEIN_PATH = '/home/ec2-user/cytoself-data/sequences.csv'
PROTEIN_EMBED_PATH_F = '/home/ec2-user/cytoself-data/ESM_sequence_embeddings_full.zarr'
PROTEIN_EMBED_PATH_R = '/home/ec2-user/cytoself-data/ESM_sequence_embeddings_reduced.zarr'

In [50]:
df = pd.read_csv(GENE2PROTEIN_PATH)
# sequences = df['Peptide']
sequences = df['Peptide'].apply(lambda x: x.replace('*', '')).values
genes = df.index.values
num_genes = len(genes)

In [51]:
import zarr


z_embedding_prot_f = zarr.open(
    PROTEIN_EMBED_PATH_F,
    mode="r"
)

z_embedding_prot_r = zarr.open(
    PROTEIN_EMBED_PATH_R,
    mode="r"
)

In [56]:
z_embedding_prot_r[0]

array([[ 0.04050937,  0.0092671 ,  0.06806058, ..., -0.24188687,
         0.19956455,  0.07856545],
       [ 0.04562815, -0.08524852,  0.00493111, ..., -0.10848487,
         0.07301189,  0.06637612]], dtype=float32)

In [58]:
z_embedding_prot_f[0, 1]

array([-0.04980165, -0.06002725,  0.03606275, ...,  0.2720689 ,
       -0.08399116,  0.01600615], dtype=float32)

In [55]:
z_embedding_prot_f

AttributeError: 'Array' object has no attribute 'max'

In [87]:
truncation_length = [min(1024, len(s)) for s in sequences]

In [64]:
truncation_length[0]

375

In [86]:
z_embedding_prot_f[0, 376]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)